In [28]:
# 1	11066	 파일 합치기
import sys

from io import StringIO
sys.stdin = StringIO('''2
4
40 30 30 50
15
1 21 3 4 5 35 5 4 3 5 98 21 14 17 32''')

def solve():
    input = sys.stdin.readline

    T = int(input())
    for _ in range(T):

        K = int(input())
        files = list(map(int, input().split()))

        S = [0] * (K + 1)  # files 0번 부터 idx 개의 챕터의 누적합
        for i in range(1, K + 1):
            S[i] = S[i-1] + files[i - 1]

        # dp[i][j] i부터 j 까지 결합하는데 최소 시간
        dp = [[0] * (K + 1) for _ in range(K + 1)]

        for length in range(2, K + 1):
            for i in range(1, K - length + 2):
                j = i + length - 1
                candi = [dp[i][k] + dp[k+1][j] for k in range(i, j)]

                # 구간을 두 부분으로 나누어 합이 최소가 되는 값 + 구간의 파일 크기 
                dp[i][j] = min(candi) + (S[j] - S[i-1])

        print(dp[1][K])
solve()

300
864


In [35]:
# 1	11066	 파일 합치기
import sys

from io import StringIO
sys.stdin = StringIO('''2
4
40 30 30 50
15
1 21 3 4 5 35 5 4 3 5 98 21 14 17 32''')

input = sys.stdin.readline

def solve():
    K = int(input())
    files = list(map(int, input().split()))

    S = [0] * (K + 1)  # files 0번 부터 idx 개의 챕터의 누적합
    for i in range(1, K + 1):
        S[i] = S[i-1] + files[i - 1]

    # dp[i][j] i부터 j 까지 결합하는데 최소 시간
    dp = [[0] * (K + 1) for _ in range(K + 1)]
    min_k = [[0] * (K + 1) for _ in range(K + 1)]

    for i in range(1, K):
        dp[i][i+1] = files[i-1] + files[i]
        min_k[i][i+1] = i # 분기점 저장

    for length in range(3, K + 1):
        for i in range(1, K - length + 2):
            j = i + length - 1

            start_k = min_k[i][j-1]
            end_k = min_k[i+1][j]

            min_val = float('inf')
            opt_k = -1

            # 크누스 최적화 적용 (증명됨) 
            for k in range(start_k, end_k + 1):
                val = dp[i][k] + dp[k+1][j]
                if val < min_val:
                    min_val = val
                    opt_k = k
            
            # 구간을 두 부분으로 나누어 합이 최소가 되는 값 + 구간의 파일 크기 
            dp[i][j] = min_val + (S[j] - S[i-1])
            min_k[i][j] = opt_k

    print(dp[1][K])

T = int(input())
for _ in range(T):
    solve()

300
864


In [ ]:
# 2	11049	 행렬 곱셈 순서
import sys

from io import StringIO
sys.stdin = StringIO('''''')

input = sys.stdin.readline

def solve():

    pass

solve()

In [ ]:
# 3	1520	 내리막 길
import sys

from io import StringIO
sys.stdin = StringIO('''''')

input = sys.stdin.readline

def solve():

    pass

solve()

In [ ]:
# 4	2629	 양팔저울
import sys

from io import StringIO
sys.stdin = StringIO('''''')

input = sys.stdin.readline

def solve():

    pass

solve()

In [ ]:
# 5	2293	 동전 1
import sys

from io import StringIO
sys.stdin = StringIO('''''')

input = sys.stdin.readline

def solve():

    pass

solve()

In [ ]:
# 6	7579	 앱
import sys

from io import StringIO
sys.stdin = StringIO('''''')

input = sys.stdin.readline

def solve():

    pass

solve()

In [ ]:
# 7	11062	 카드 게임
import sys

from io import StringIO
sys.stdin = StringIO('''''')

input = sys.stdin.readline

def solve():

    pass

solve()